<a href="https://colab.research.google.com/github/Beh-noush/General-Python/blob/main/FCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple Neural Network with Pytorch

In [1]:
import torch
import torch.nn as nn
import torch.optim
import torch.nn.functional as F

In [6]:
from sklearn import datasets

A fully-connected 2-layer network with Relu non-linearity.

In [2]:
class FCNN(nn.Module):
  def __init__(self, input_dim, hidden_dim, out_dim):
    super(FCNN, self).__init__()
    self.preactivation_hidden = nn.Linear(input_dim,hidden_dim)
   # self.activation_hidden = nn.ReLU(self.preactivation_hidden)
    self.preactivation_out = nn.Linear(hidden_dim, out_dim)
  #  self.activation_hidden = nn.ReLU(self.preactivation_out)

  def forward(self, x):
    """
    Input:
          1-d array
          x is the input of the Neural Net of dimension input_dim
    Returns:
          1-d array of dimension out_dim. The score function output.
    """
    preactive_1 = self.preactivation_hidden(x)
    active_1 = F.relu(preactive_1)
    pre_output = self.preactivation_out(active_1)
    output = F.relu(pre_output)
    return output

# Why is this code wrong? 
#Ok, By replacing nn.ReLU with F.relu it works.
#It remains to check that both network structures are correct.


In [3]:
class FCNN(nn.Module):
  def __init__(self, input_dim, hidden_dim, out_dim):
    super(FCNN, self).__init__()
    self.preactivation_hidden = nn.Linear(input_dim,hidden_dim)
    self.activation_hidden = nn.ReLU()
    self.preactivation_out = nn.Linear(hidden_dim, out_dim)
    self.activation_out = nn.Softmax()

  def forward(self, x):
    """
    Input:
          1-d array
          x is the input of the Neural Net of dimension input_dim
    Returns:
          1-d array of dimension out_dim. The score function output.
    """
    preactive_1 = self.preactivation_hidden(x)
    active_1 = self.activation_hidden(preactive_1)
    pre_output = self.preactivation_out(active_1)
    output = self.activation_out(pre_output)
    return output



We use this neural net on Iris dataset.

In [7]:
iris = datasets.load_iris()

In [8]:
type(iris['data'])

numpy.ndarray

In [9]:
iris_data_torch = torch.from_numpy(iris.data)

In [12]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

This data is now ready to be feed into the neeural net.


In [10]:
my_net = FCNN(4, 10, 3) 

Remember it was necessary to put .float() after your dataset, otherwise an error occured complaining about the dtype double rather than float! It seems a new error!

In [17]:
output = my_net.forward(iris_data_torch.float())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Now, we can train our neural net on this dataset.

In [20]:
list(my_net.parameters()) #It contains weights and biases.

[Parameter containing:
 tensor([[-0.1896,  0.2431, -0.1592,  0.1351],
         [-0.1103, -0.0021, -0.4792, -0.1542],
         [ 0.3189,  0.1173,  0.0658,  0.4400],
         [-0.3774, -0.3951, -0.0963, -0.4424],
         [-0.4285, -0.0937,  0.0095,  0.1947],
         [ 0.4938,  0.3886,  0.1496,  0.3671],
         [ 0.2349, -0.0159,  0.0377,  0.0407],
         [ 0.0577, -0.3281, -0.2493, -0.3240],
         [ 0.4118, -0.1384,  0.1005, -0.2762],
         [-0.0984,  0.0507,  0.0297,  0.4482]], requires_grad=True),
 Parameter containing:
 tensor([-0.2940, -0.1789,  0.1031,  0.0311, -0.3952,  0.3669,  0.3276,  0.0612,
          0.0241, -0.3591], requires_grad=True),
 Parameter containing:
 tensor([[ 0.2703, -0.2946,  0.0445, -0.2724, -0.1111,  0.0457, -0.1401,  0.0805,
          -0.1148,  0.2158],
         [ 0.0563, -0.0449, -0.2378, -0.0763, -0.0767, -0.1795,  0.2799, -0.0516,
           0.0442,  0.1511],
         [-0.1802,  0.3110, -0.2072,  0.0714,  0.2499,  0.0374,  0.2495, -0.0244,
     

In [19]:
loss = nn.CrossEntropyLoss()
iris_labels_torch = torch.from_numpy(iris.target)

loss(output[:10], iris_labels_torch[:10])

tensor(0.9658, grad_fn=<NllLossBackward>)

In [21]:
torch.argmax(output, axis = 1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])

In [25]:
#First, we train with unnormalized data.
epochs = range(20)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(my_net.parameters(), lr=1e-1)
iris_train = iris_data_torch.float()
iris_labels_torch = torch.from_numpy(iris.target)
iris_train.requires_grad = True
for e in epochs:
   optimizer.zero_grad()
   output = my_net.forward(iris_train)
   predictions = torch.argmax(output, axis = 1)
   iris_loss = loss(output, iris_labels_torch)
   iris_loss.backward()
   optimizer.step()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [26]:
predictions

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
        2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

In [27]:
list(my_net.parameters()) 

[Parameter containing:
 tensor([[-0.1896,  0.2431, -0.1592,  0.1351],
         [-0.1103, -0.0021, -0.4792, -0.1542],
         [-0.1247, -0.0710, -0.3969, -0.0034],
         [-0.3774, -0.3951, -0.0963, -0.4424],
         [-0.4285, -0.0937,  0.0095,  0.1947],
         [-0.0624, -0.5177,  1.0792,  2.1012],
         [-0.2646, -1.0768,  0.8852,  1.4750],
         [ 0.0577, -0.3281, -0.2493, -0.3240],
         [-0.2500, -0.9706, -0.4718, -0.9792],
         [-0.6287, -0.4797, -0.5006, -0.0821]], requires_grad=True),
 Parameter containing:
 tensor([-0.2940, -0.1789, -0.3357,  0.0311, -0.3952, -0.7326, -0.6881,  0.0612,
         -0.5110, -0.8894], requires_grad=True),
 Parameter containing:
 tensor([[ 0.2703, -0.2946, -0.4506, -0.2724, -0.1111, -2.0650, -1.4811,  0.0805,
          -0.2641, -0.3145],
         [ 0.0563, -0.0449,  0.0943, -0.0763, -0.0767,  0.7417, -1.3729, -0.0516,
           0.4781, -0.3792],
         [-0.1802,  0.3110,  0.3501,  0.0714,  0.2499,  0.5196,  2.3792, -0.0244,
     

In [34]:
(predictions==iris_labels_torch).float().mean()

tensor(0.9800)